In [31]:
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
df_2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')
df_2001 = pd.read_csv('Data/final_tmdb_data_2001.csv.gz')
df = pd.concat([df_2000,df_2001] )

In [33]:
Pgrating= df.loc[df['certification']=='PG',"revenue"]
Pg13rating= df.loc[df['certification']=='PG-13',"revenue"]
Rrating=df.loc[df['certification']=='R',"revenue"]
Grating=df.loc[df['certification']=='G',"revenue"]
NRrating=df.loc[df['certification']=='NR',"revenue"]

In [34]:
print(f"For PG ratings(n={len(Pgrating)}): Mean={np.mean(Pgrating):.2f}")
print(f"For PG-13 ratings(n={len(Pg13rating)}): Mean={np.mean(Pg13rating):.2f}")
print(f"For R ratings(n={len(Rrating)}): Mean={np.mean(Rrating):.2f}")
print(f"For G ratings(n={len(Grating)}): Mean={np.mean(Grating):.2f}")
print(f"For NR ratings(n={len(NRrating)}): Mean={np.mean(rating):.2f}")

For PG ratings(n=63): Mean=62583103.89
